<a href="https://colab.research.google.com/github/hyunicecream/ML-DL/blob/main/Majority_Voting(Diabetes_Data).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import  classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
import numpy as np

In [ ]:
# 데이터를 불러온다.
diabetes = load_diabetes()

In [ ]:
diabetes.keys()

dict_keys(['data', 'target', 'frame', 'DESCR', 'feature_names', 'data_filename', 'target_filename', 'data_module'])

In [ ]:
# 궁금해서 뭐가 들었는지 보자
# 데이터 프레임으로 만들어서 확인해보기
import pandas as pd

df =  pd.DataFrame(diabetes.data, columns=[diabetes.feature_names])
df

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641
...,...,...,...,...,...,...,...,...,...,...
437,0.041708,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,0.007207
438,-0.005515,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018118,0.044485
439,0.041708,0.050680,-0.015906,0.017282,-0.037344,-0.013840,-0.024993,-0.011080,-0.046879,0.015491
440,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044528,-0.025930


In [ ]:
trainX, testX, trainY, testY = train_test_split(diabetes['data'], diabetes['target'], test_size = 0.2)

In [ ]:
# 4가지 모델로 구성
# 각 모델은 최적 조건으로 생성한다. (knn의 k개수, dtree의 max_depth 등)
# sklearn 문서 : Recommended for an ensemble of well-calibrated classifiers.
knn = KNeighborsClassifier(n_neighbors = 5, p = 2, metric='minkowski')
dtree = DecisionTreeClassifier(criterion= 'gini', max_depth=8)
svm = SVC(kernel='rbf', gamma=0.1, C=1.0, probability=True)
lreg = LogisticRegression(max_iter=500)

In [ ]:
# 4가지 모델로 앙상블을 구성한다.
# 4가지 모델을 각각 학습하고, 결과를 종합한다.
# VotingClassifier()의 voting = ('hard' or 'soft')에 따라 아래와 같이 종합한다.
# hard (default) : 4가지 모델에서 추정한 class = (0, 1, 2)중 가장 많은 것으로 판정.
# soft : 4가지 모델에서 추정한 각 class의 확률값의 평균 (혹은 합)을 계산한 후,
# 확률이 가장 높은 (argmax(P)) class로 판정한다.
base_model = [('knn', knn), ('dtree', dtree), ('svm', svm), ('lr', lreg)]
ensemble = VotingClassifier(estimators=base_model, voting='soft')
ensemble.fit(testX, testY)

VotingClassifier(estimators=[('knn', KNeighborsClassifier()),
                             ('dtree', DecisionTreeClassifier(max_depth=8)),
                             ('svm', SVC(gamma=0.1, probability=True)),
                             ('lr', LogisticRegression(max_iter=500))],
                 voting='soft')

In [ ]:
# 시험데이터의 정확도를 측정한다.
print('시험 데이터의 정확도 = %.2f' % ensemble.score(testX, testY))

시험 데이터의 정확도 = 0.82


In [ ]:
# 시험데이터의 confusion matrix를 작성하고, (row : actual, col : predict),
# 4개 score를 확인한다.
predY = ensemble.predict(testX)
print('\nConfusion matrix :')
print(confusion_matrix(testY, predY))
# print(classification_report(testY, predY))


Confusion matrix :
[[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 2 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]]


In [ ]:
# 표준화 시켜서 전과 비교해보기
from sklearn.preprocessing import StandardScaler

feature_data = diabetes['data']
feature_data = StandardScaler().fit_transform(feature_data)

In [ ]:
trainX, testX, trainY, testY = train_test_split(diabetes['data'], diabetes['target'], test_size = 0.2)
# 4가지 모델로 구성
knn = KNeighborsClassifier(n_neighbors = 5, p = 2, metric='minkowski')
dtree = DecisionTreeClassifier(criterion= 'gini', max_depth=8)
svm = SVC(kernel='rbf', gamma=0.1, C=1.0, probability=True)
lreg = LogisticRegression(max_iter=100)
# 4가지 모델로 앙상블을 구성한다.
base_model = [('knn', knn), ('dtree', dtree), ('svm', svm), ('lr', lreg)]
ensemble = VotingClassifier(estimators=base_model, voting='soft')
ensemble.fit(testX, testY)
# 학습데이터와 시험데이터의 정확도를 측정한다.
print('시험데이터의 정확도 = %.2f' % ensemble.score(testX, testY))

시험데이터의 정확도 = 0.91


In [ ]:
# 결과 : 오락가락 한다.